In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
import random as rd
from sklearn.model_selection import train_test_split 

from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical  
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD


import warnings
warnings.filterwarnings("ignore")


digits = load_digits()

In [16]:
dataset = pd.read_csv('https://s3.us-west-2.amazonaws.com/public.gamelab.fun/dataset/position_salaries.csv')
X = dataset.iloc[:, 1:2].values
y = dataset.iloc[:, 2].values



# Polynomial Regression

The code in the cell below generates the linear data model using the polynomial regression algorithm. The function takes in the entire training set along with a single test example x (x does not have to be in training set) and the value a which specifies the percentage of the training set we want to use as subsets.

In [17]:
def estimateDM_PR(x, x_train, y_train, a):
    m = 10000
    size = round(a * len(y_train))
    T_x = np.zeros(shape=(m, x_train.shape[0]))
    T_y = np.zeros(shape=(m,1))
    for i in range(m):
        # sample subset si of S where |si|= a|S| 
        #print(size)
        count = 0
        ind = [0] * len(y_train)
        t = []
        s = np.zeros(shape=(size,x_train.shape[1]))
        z = np.zeros(shape=(size,1))
        
        # generate random list of indexes
        indexes = rd.sample(range(x_train.shape[0]), size)
        for index in indexes:
            ind[index] = 1
            s[count] = x_train[index]
            z[count] = y_train[index]
            count+=1
                
        
        #train model  
        #logisticRegr = LogisticRegression()
        #logisticRegr.fit(s, z)
        #val = logisticRegr.predict(x)
        #print(val)
        poly_reg = PolynomialFeatures(degree=4)
        X_poly = poly_reg.fit_transform(s)
        pol_reg = LinearRegression()
        pol_reg.fit(X_poly, z)
        
        val = pol_reg.predict(poly_reg.fit_transform([[x]]))
        #print(val)
        T_x[i] = ind
        #T_y[i] = val[0]
        T_y[i] = val
    theta = runRegression(T_x,T_y,x)
    #print(theta.shape)
    return theta
         
    
def runRegression(T_x,T_y,x):
    #print(T_x)
    #print(T_y)
    lasso = Lasso(alpha=1)
    #lasso = linear_model.Lasso(alpha=0.1)
    lasso.fit(T_x, T_y)
    #print(lasso.predict(T_x))
    #print(lasso.coef_)
    return lasso




In [19]:
ldm = estimateDM_PR(15, X, y, 0.3)

Here we verify the accuracy of the linear data model using alpha = 0.3 and x = 15 by comparing it to the actual results of training a poly reg model with the entire training set and evauluating at the same value for x

In [20]:
# verification poly reg
m = 1
a = 0.3
size = round(a * len(y))
x = 15
for i in range(m):
    count = 0
    ind = [0] * len(y)
    t = []
    s = np.zeros(shape=(size,X.shape[1]))
    z = np.zeros(shape=(size,1))
        
    # generate random list of indexes
    indexes = rd.sample(range(X.shape[0]), size)
    for index in indexes:
        ind[index] = 1
        s[count] = X[index]
        z[count] = y[index]
        count+=1
    
    
    # linear model prediction
    prediction = ldm.predict([ind])
    
    # actual model
    poly_reg = PolynomialFeatures(degree=4)
    X_poly = poly_reg.fit_transform(s)
    pol_reg = LinearRegression()
    pol_reg.fit(X_poly, z)
    
    actual = pol_reg.predict(poly_reg.fit_transform([[x]]))
    
    # percentage difference
    print(ind)
    print("Prediction: " + str(prediction))
    print("Actual: " + str(actual))
    pdiff = (abs(prediction - actual)/actual) * 100
    print("% diff" + str(pdiff))
    
    

[0, 0, 0, 1, 0, 0, 1, 1, 0, 0]
Prediction: [3076728.94235819]
Actual: [[3220757.60132571]]
% diff[[4.47188757]]


for the polynomial model with datset of 10 features, ldm performs extremely 
well for all possible datasets for alpha > 0.4. Accuracy dips when alpha < 0.3 an varies from being very 
accurate to not accurate at all.

# KNN

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets

In [22]:
#iris = datasets.load_iris()
#X = iris.data[:, :4]  
#y = iris.target

cancer = datasets.load_breast_cancer()
X = cancer.data[:, :39]
y = cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Following the same format the code below generates a linear datamodel using the KNN algorithm. We pass in the same parameters

In [23]:
def estimateDM_KNN(x, x_train, y_train, a):
    m = 1000
    size = round(a * len(y_train))
    T_x = np.zeros(shape=(m, x_train.shape[0]))
    T_y = np.zeros(shape=(m,1))
    for i in range(m):
        # sample subset si of S where |si|= a|S| 
        #print(size)
        count = 0
        ind = [0] * len(y_train)
        t = []
        s = np.zeros(shape=(size,x_train.shape[1]))
        z = np.zeros(shape=(size,1))
        
        # generate random list of indexes
        indexes = rd.sample(range(x_train.shape[0]), size)
        for index in indexes:
            ind[index] = 1
            s[count] = x_train[index]
            z[count] = y_train[index]
            count+=1
                
        
        #train model  
        neigh = KNeighborsClassifier(n_neighbors=3)
        neigh.fit(s, z)
        
        y = neigh.predict([x])
        #y = neigh.predict_proba([x])
        #print(y)

        #corr_log = y[0][y_indx]
        #y = np.delete(y, y_indx)
        #incorr_log = max(y)
        
        T_x[i] = ind
        T_y[i] = y
    lasso = runRegression(T_x,T_y,x)
    #print(theta.shape)
    return lasso
    
def runRegression(T_x,T_y,x):
    #print(T_x)
    #print(T_y)
    lasso = Lasso(alpha=1)
    #lasso = linear_model.Lasso(alpha=0.1)
    lasso.fit(T_x, T_y)
    #print(lasso.predict(T_x))
    #print(lasso.coef_)
    return lasso


In [ ]:
ldm_knn = estimateDM_KNN(X_test[0], X_train, y_train, 0.8)

Verification of actual predictions vs linear datamodel predictions. We run the linear datamodel algorithm against multiple values of X and vary the value of alpha. The results are detailed below

In [30]:
# verification knn
#m = 100
a = 0.7
size = round(a * len(y_train))
corr_count = 0
for i in range(len(X_test)):
    x = X_test[i]
    ldm_knn = estimateDM_KNN(x, X_train, y_train, a)
    count = 0
    ind = [0] * len(y_train)
    t = []
    s = np.zeros(shape=(size,X_train.shape[1]))
    z = np.zeros(shape=(size,1))
        
    # generate random list of indexes
    indexes = rd.sample(range(X_train.shape[0]), size)
    for index in indexes:
        ind[index] = 1
        s[count] = X_train[index]
        z[count] = y_train[index]
        count+=1
    
    
    # linear model prediction
    prediction = ldm_knn.predict([ind])
    
    # actual model
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(s, z)
    
    actual = neigh.predict([x])
    
    if int(round(prediction[0], 0)) == int(actual[0]):
        corr_count += 1
    else:
        print(ind)
        print("Prediction: " + str(prediction[0]))
        print("Actual: " + str(actual[0]))
    # percentage difference
    #print(ind)
    #print("Prediction: " + str(prediction[0]))
    #print("Actual: " + str(actual[0]))
    #pdiff = (abs(prediction - actual)/actual) * 100
    #print("% diff" + str(pdiff))
print("Accuracy: " + str(corr_count/len(X_test)))
    
    

[0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 

RESULTS
iris data set, 4 features, 150 total points

results for a test set of 50 samples, and a train set of 100 samples
at a = 0.2, acc = 0.94
at a = 0.4, acc = 0.98
at a = 0.6, acc = 0.98
at a = 0.8, acc = 1.0

results for a test set of 90 samples, and a train set of 60 samples
at a = 0.2, acc = 0.91
at a = 0.4, acc = 0.95
at a = 0.6, acc = 0.97


cancer data set, 39 features, 569 total points

results for test set of 114 and train set of 455
at a = 0.5, acc = 0.98
at a = 0.3, acc = 0.96
at a = 0.8, acc = 0.97

# NN

In [81]:

def estimateDataModel(x, y_indx, x_train, y_train, a):
    
    m = 1
    size = round(a * len(y_train))
    T_x = np.zeros(shape=(m, x_train.shape[0]))
    T_y = np.zeros(shape=(m,1))
    for i in range(m):
        # sample subset si of S where |si|= a|S| 
        #print(size)
        count = 0
        ind = [0] * len(y_train)
        t = []
        s = np.zeros(shape=(size,x_train.shape[1]))
        z = np.zeros(shape=(size,10))
        
        # generate random list of indexes
        indexes = rd.sample(range(x_train.shape[0]), size)
        for index in indexes:
            ind[index] = 1
            s[count] = x_train[index]
            z[count] = y_train[index]
            count+=1
                
        
        #train model  
        
        #train model
        model = Sequential()
        model.add(Dense(256, activation='relu', input_dim=3072))
        model.add(Dense(256, activation='relu'))
        model.add(Dense(10, activation='softmax'))
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

        model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
        print(x_train.shape)
        print(s.shape)
        history = model.fit(s,z, epochs=10, batch_size=32, verbose=2, validation_split=0.2)
        y = model.predict(x)
        corr_log = y[0][y_indx]
        y = np.delete(y, y_indx)
        incorr_log = max(y)
        #print(y.shape)
        
        #ind_y = np.zeros(shape=(len(S),3072))
        #for i in range(len(S)):
        #if ind[i] == 1:
         #       ind_y[i] = S[i]
        
        #T.append([y,ind_y])
        T_x[i] = ind
        T_y[i] = corr_log - incorr_log
    lasso = runRegression(T_x,T_y)
    #print(theta.shape)
    return lasso
         
    
def runRegression(T_x,T_y):
    lasso = Lasso(alpha=0.001)
    lasso.fit(T_x, T_y)
    print(lasso.coef_)
    return lasso
            
    

In [184]:
def estimateDataModel_NN(x, x_train, y_train, a):
    
    m = 500
    size = round(a * len(y_train))
    T_x = np.zeros(shape=(m, x_train.shape[0]))
    T_y = np.zeros(shape=(m,1))
    for i in range(m):
        # sample subset si of S where |si|= a|S| 
        #print(size)
        count = 0
        ind = [0] * len(y_train)
        t = []
        s = np.zeros(shape=(size,x_train.shape[1]))
        z = np.zeros(shape=(size,1))
        
        #print(s.shape)
        #print(z.shape)
        
        # generate random list of indexes
        indexes = rd.sample(range(x_train.shape[0]), size)
        for index in indexes:
            ind[index] = 1
            s[count] = x_train[index]
            z[count] = y_train[index]
            count+=1
                
        
        #train model  
        s = torch.tensor(s, dtype=torch.float)
        z = torch.tensor(z, dtype=torch.float)
        model = nn.Sequential(nn.Linear(30, 10),
                      nn.ReLU(),
                      nn.Linear(10, 1),
                      nn.Sigmoid())
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
        for epoch in range(1000):
            pred_y = model(s)
            loss = criterion(pred_y, z)

            model.zero_grad()
            loss.backward()

            optimizer.step()
            
            
        y = model(x)
        #print(y)
        
        T_x[i] = ind
        T_y[i] = np.array(y.detach())
        
    lasso = runRegression(T_x,T_y)
    #print(theta.shape)
    return lasso
         
    
def runRegression(T_x,T_y):
    lasso = Lasso(alpha=0.001)
    lasso.fit(T_x, T_y)
    return lasso
            
    

In [119]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Transform label indices to one-hot encoded vectors

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Transform images from (32,32,3) to 3072-dimensional vectors (32*32*3)

X_train = np.reshape(X_train,(50000,3072))
X_test = np.reshape(X_test,(10000,3072))
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalization of pixel values (to [0-1] range)

X_train /= 255
X_test /= 255

In [134]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn


data = load_breast_cancer()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [168]:
y = y_test[0]

y_test = np.reshape(y_test, (len(y_test), 1))
y_train = np.reshape(y_train, (len(y_train), 1))
#print(y_test.shape)
#print(y_train.shape)
x = np.reshape(X_test[0], (1, 30))
x = torch.tensor(x, dtype=torch.float)


lasso = estimateDataModel_NN(x, X_train, y_train, 0.8)


In [179]:
size = round(a * len(y_train))
x = X_test[0]
ind = [0] * len(y_train)
t = []
s = np.zeros(shape=(size,X_train.shape[1]))
z = np.zeros(shape=(size,1))

# generate random list of indexes
indexes = rd.sample(range(X_train.shape[0]), size)
count = 0
for index in indexes:
    ind[index] = 1
    s[count] = X_train[index]
    z[count] = y_train[index]
    count+=1
    
    
# linear model prediction
prediction = lasso.predict([ind])
print(prediction)
print(actual)

actual = y_test[0]
    
if int(round(prediction[0], 0)) == int(actual[0]):
    corr_count += 1
else:
    print(ind)
    print("Prediction: " + str(prediction[0]))
    print("Actual: " + str(actual[0]))
    

[0.71668254]
[1]


In [186]:
# verification nn
#m = 100
loc = []
vals = []
a = 0.5
size = round(a * len(y_train))
corr_count = 0
for i in range(len(X_test)):
    x = X_test[i]
    x = np.reshape(X_test[0], (1, 30))
    x = torch.tensor(x, dtype=torch.float)
    ldm_nn = estimateDataModel_NN(x, X_train, y_train, a)
    count = 0
    ind = [0] * len(y_train)
    t = []
    s = np.zeros(shape=(size,X_train.shape[1]))
    z = np.zeros(shape=(size,1))
        
    # generate random list of indexes
    indexes = rd.sample(range(X_train.shape[0]), size)
    for index in indexes:
        ind[index] = 1
        s[count] = X_train[index]
        z[count] = y_train[index]
        count+=1
    
    
    # linear model prediction
    prediction = ldm_nn.predict([ind])
    
    # actual model
    #neigh = KNeighborsClassifier(n_neighbors=3)
    #neigh.fit(s, z)
    
    actual = y_test[i]
    
    if int(round(prediction[0], 0)) == int(actual[0]):
        corr_count += 1
    else:
        print(ind)
        print("Index", i)
        print("Prediction: " + str(prediction[0]))
        print("Actual: " + str(actual[0]))
        loc.append(i)
        vals.append(ind)
    # percentage difference
    #print(ind)
    #print("Prediction: " + str(prediction[0]))
    #print("Actual: " + str(actual[0]))
    #pdiff = (abs(prediction - actual)/actual) * 100
    #print("% diff" + str(pdiff))
print("Accuracy: " + str(corr_count/len(X_test)))

[1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 

[0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 

[1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 

In [187]:
print(loc)

[1, 2, 6, 7, 11, 13, 15, 16, 19, 20, 22, 29, 36, 38, 42, 50, 51]


In [198]:
#get bad data and remove
vhist = [0] * len(X_train) 
for l in vals:
    #print(len(l))
    vhist = [sum(x) for x in zip(vhist, l)]
    
id_list = []
c = 0
for v in vhist:
    if v > 10:
        id_list.append(c)
    c += 1
    
print(id_list)  

512


In [211]:
new_len = len(X_train) - len(id_list)
n_X_train = np.zeros(shape=(new_len, X_train.shape[1]))
n_y_train = np.zeros(shape=(new_len, y_train.shape[1]))


c = 0
for i in range(len(X_train)):
    if (i not in id_list):
        n_X_train[c] = X_train[i]
        n_y_train[c] = y_train[i]
        c += 1
 

In [213]:
loc_n = []
vals_n = []
a = 0.5
size = round(a * len(n_y_train))
corr_count = 0
for i in range(len(X_test)):
    x = X_test[i]
    x = np.reshape(X_test[0], (1, 30))
    x = torch.tensor(x, dtype=torch.float)
    ldm_nn = estimateDataModel_NN(x, n_X_train, n_y_train, a)
    count = 0
    ind = [0] * len(n_y_train)
    t = []
    s = np.zeros(shape=(size,n_X_train.shape[1]))
    z = np.zeros(shape=(size,1))
        
    # generate random list of indexes
    indexes = rd.sample(range(n_X_train.shape[0]), size)
    for index in indexes:
        ind[index] = 1
        s[count] = n_X_train[index]
        z[count] = n_y_train[index]
        count+=1
    
    
    # linear model prediction
    prediction = ldm_nn.predict([ind])
    
    # actual model
    #neigh = KNeighborsClassifier(n_neighbors=3)
    #neigh.fit(s, z)
    
    actual = y_test[i]
    
    if int(round(prediction[0], 0)) == int(actual[0]):
        corr_count += 1
    else:
        print(ind)
        print("Index", i)
        print("Prediction: " + str(prediction[0]))
        print("Actual: " + str(actual[0]))
        loc_n.append(i)
        vals_n.append(ind)
    # percentage difference
    #print(ind)
    #print("Prediction: " + str(prediction[0]))
    #print("Actual: " + str(actual[0]))
    #pdiff = (abs(prediction - actual)/actual) * 100
    #print("% diff" + str(pdiff))
print("Accuracy: " + str(corr_count/len(X_test)))

[0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 

[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 

[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 